In [8]:
from tqdm.auto import tqdm
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import re

In [9]:
df1 = pd.read_csv("/Users/julianalexander/Library/Mobile Documents/com~apple~CloudDocs/HSG/1. Semester MBI FS2021/8,049,1.00 Text Mining mit Python/TextMining21/Reddit_Julian_TOTAL-klassifiziert.csv")
df2 = pd.read_csv("/Users/julianalexander/Library/Mobile Documents/com~apple~CloudDocs/HSG/1. Semester MBI FS2021/8,049,1.00 Text Mining mit Python/TextMining21/Reddit_Marek_klassifiziert.csv")

In [10]:
Atweets = pd.concat([df1, df2], axis=0, join='inner')
Atweets = Atweets.sort_values(by='index')
Atweets = Atweets.reset_index(drop=True)

Punktuierung

In [17]:
def remove_punctuation(txt):
    txt_nopunt = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunt

In [18]:
Atweets["submission"] = Atweets["submission"].apply(lambda x: remove_punctuation(x))

Tokenisierung

In [20]:
def tokenize(txt):
    tokens = re.split("\W+", txt)
    return tokens

In [22]:
Atweets["submissions_tokenized"] = Atweets["submission"].apply(lambda x: tokenize(x.lower()))

Stopwörter

In [24]:
stopwords = nltk.corpus.stopwords.words("english")

In [25]:
def remove_stopwords(txt_tokenized):
    txt_clean = [word for word in txt_tokenized if word not in stopwords]
    return txt_clean

In [26]:
Atweets["submission_no_stopwords"] = Atweets["submissions_tokenized"].apply(lambda x: remove_stopwords(x))

Text-Bereinigung

In [29]:
def clean_text(text):
    text = "".join([c for c in text if c not in string.punctuation])
    tokens = re.split("\W+", text)
    text = [word for word in tokens if word not in stopwords]
    return text

In [30]:
Atweets["submission_nostop"] = Atweets["submission"].apply(lambda x: clean_text(x.lower()))

Stemming

In [35]:
ps = PorterStemmer()

In [36]:
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

In [37]:
Atweets["submission_stemmed"] = Atweets["submission_nostop"].apply(lambda x: stemming(x))

Lemmatisierung

In [39]:
wn = nltk.WordNetLemmatizer()

In [40]:
def lemmatization(token_txt):
    text = [wn.lemmatize(word) for word in token_txt]
    return text

In [47]:
Atweets["submission_lemmatized"] = Atweets["submission_nostop"].apply(lambda x: lemmatization(x))

In [48]:
Atweets

,index,submission,sentiment,submissions_tokenized,submission_no_stopwords,submission_nostop,submission_stemmed,submission_lemmatized
0,0,I see that many countries are working hard to ...,2.0,"[i, see, that, many, countries, are, working, ...","[see, many, countries, working, hard, publish,...","[see, many, countries, working, hard, publish,...","[see, mani, countri, work, hard, publish, news...","[see, many, country, working, hard, publish, n..."
1,0,httpwwwtheguardiancomtechnology2013oct29bitcoi...,3.0,[httpwwwtheguardiancomtechnology2013oct29bitco...,[httpwwwtheguardiancomtechnology2013oct29bitco...,[httpwwwtheguardiancomtechnology2013oct29bitco...,[httpwwwtheguardiancomtechnology2013oct29bitco...,[httpwwwtheguardiancomtechnology2013oct29bitco...
2,1,so like many im not 100 clear on how LN functi...,3.0,"[so, like, many, im, not, 100, clear, on, how,...","[like, many, im, 100, clear, ln, functions, gr...","[like, many, im, 100, clear, ln, functions, gr...","[like, mani, im, 100, clear, ln, function, gra...","[like, many, im, 100, clear, ln, function, gra..."
3,1,Because I see a lot of down and out people who...,2.0,"[because, i, see, a, lot, of, down, and, out, ...","[see, lot, people, fucked, loanswhat, think, r...","[see, lot, people, fucked, loanswhat, think, r...","[see, lot, peopl, fuck, loanswhat, think, rain...","[see, lot, people, fucked, loanswhat, think, r..."
4,2,I started the sync a few days ago and it was f...,2.0,"[i, started, the, sync, a, few, days, ago, and...","[started, sync, days, ago, fine, first, going,...","[started, sync, days, ago, fine, first, going,...","[start, sync, day, ago, fine, first, go, 46, e...","[started, sync, day, ago, fine, first, going, ..."
...,...,...,...,...,...,...,...,...
1998,2594,I know there are a lot of walk throughs about ...,3.0,"[i, know, there, are, a, lot, of, walk, throug...","[know, lot, walk, throughs, getting, private, ...","[know, lot, walk, throughs, getting, private, ...","[know, lot, walk, through, get, privat, key, s...","[know, lot, walk, throughs, getting, private, ..."
1999,2595,how is a cryptocurrency exchange different tha...,3.0,"[how, is, a, cryptocurrency, exchange, differe...","[cryptocurrency, exchange, different, bank, le...","[cryptocurrency, exchange, different, bank, le...","[cryptocurr, exchang, differ, bank, less, prof...","[cryptocurrency, exchange, different, bank, le..."
2000,2596,A US federal judge rejected Alibaba Group Hold...,3.0,"[a, us, federal, judge, rejected, alibaba, gro...","[us, federal, judge, rejected, alibaba, group,...","[us, federal, judge, rejected, alibaba, group,...","[us, feder, judg, reject, alibaba, group, hold...","[u, federal, judge, rejected, alibaba, group, ..."
2001,2597,Anyone know of any Also do most online shops a...,3.0,"[anyone, know, of, any, also, do, most, online...","[anyone, know, also, online, shops, accept, lo...","[anyone, know, also, online, shops, accept, lo...","[anyon, know, also, onlin, shop, accept, lot, ...","[anyone, know, also, online, shop, accept, lot..."
